# Extract Bottleneck Features for Train Set

In [1]:
import keras
from keras.datasets import cifar10
import numpy as np
from keras.applications.inception_v3 import InceptionV3, preprocess_input
import scipy
from scipy import misc
import os
from keras.callbacks import ModelCheckpoint   
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import MaxPooling2D
from keras.layers.normalization import BatchNormalization

# load the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = np.squeeze(y_train)
print('data loaded')

# load inceptionV3 model + remove final classification layers
model = InceptionV3(weights='imagenet', include_top=False, input_shape=(139, 139, 3))
print('model loaded')

# obtain bottleneck features (train)
if os.path.exists('inception_features_train.npz'):
    print('bottleneck features detected (train)')
    features = np.load('inception_features_train.npz')['features']
else:
    print('bottleneck features file not detected (train)')
    print('calculating now ...')
    # pre-process the train data
    big_x_train = np.array([scipy.misc.imresize(x_train[i], (139, 139, 3)) 
                            for i in range(0, len(x_train))]).astype('float32')
    inception_input_train = preprocess_input(big_x_train)
    print('train data preprocessed')
    # extract, process, and save bottleneck features
    features = model.predict(inception_input_train)
    features = np.squeeze(features)
    np.savez('inception_features_train', features=features)
print('bottleneck features saved (train)')

/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


data loaded
model loaded
bottleneck features detected (train)
bottleneck features saved (train)


# Extract Bottleneck Features for Test Set

In [2]:
# obtain bottleneck features (test)
if os.path.exists('inception_features_test.npz'):
    print('bottleneck features detected (test)')
    features_test = np.load('inception_features_test.npz')['features_test']
else:
    print('bottleneck features file not detected (test)')
    print('calculating now ...')
    # pre-process the test data
    big_x_test = np.array([scipy.misc.imresize(x_test[i], (139, 139, 3)) 
                       for i in range(0, len(x_test))]).astype('float32')
    inception_input_test = preprocess_input(big_x_test)
    # extract, process, and save bottleneck features (test)
    features_test = model.predict(inception_input_test)
    features_test = np.squeeze(features_test)
    np.savez('inception_features_test', features_test=features_test)
print('bottleneck features saved (test)')

bottleneck features detected (test)
bottleneck features saved (test)


# Train a Shallow NN

In [3]:
from keras.utils import np_utils

# one-hot encode the labels
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [5]:
features.shape[1:]

(3, 3, 2048)

In [7]:
model = model2()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_95 (Conv2D)           (None, 2, 2, 128)         1048704   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 2, 128)         0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
__________

In [8]:
batch_size = 256
epochs = 50

checkpointer = ModelCheckpoint(filepath='improvement-{epoch:02d}-{val_acc:.2f}.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(features, y_train, batch_size=batch_size, epochs=epochs,
          validation_split=0.1, callbacks=[checkpointer],
          verbose=2, shuffle=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
 - 5s - loss: 2.3836 - acc: 0.3188 - val_loss: 1.1226 - val_acc: 0.6834

Epoch 00001: val_loss improved from inf to 1.12255, saving model to improvement-01-0.68.hdf5
Epoch 2/50
 - 3s - loss: 1.4268 - acc: 0.5839 - val_loss: 0.9846 - val_acc: 0.7276

Epoch 00002: val_loss improved from 1.12255 to 0.98458, saving model to improvement-02-0.73.hdf5
Epoch 3/50
 - 3s - loss: 1.1256 - acc: 0.6738 - val_loss: 0.7512 - val_acc: 0.7854

Epoch 00003: val_loss improved from 0.98458 to 0.75117, saving model to improvement-03-0.79.hdf5
Epoch 4/50
 - 3s - loss: 0.9848 - acc: 0.7083 - val_loss: 0.6797 - val_acc: 0.7928

Epoch 00004: val_loss improved from 0.75117 to 0.67972, saving model to improvement-04-0.79.hdf5
Epoch 5/50
 - 3s - loss: 0.8546 - acc: 0.7432 - val_loss: 0.6601 - val_acc: 0.8008

Epoch 00005: val_loss improved from 0.67972 to 0.66010, saving model to improvement-05-0.80.hdf5
Epoch 6/50
 - 3s - loss: 0.7831 - acc: 0.7592 - va

In [9]:
from glob import glob

for g in glob('*.hdf5'):
    
    model.load_weights(g)

    # evaluate test accuracy
    score = model.evaluate(features_test, y_test, verbose=0)
    accuracy = 100*score[1]

    # print test accuracy
    print('%s Test accuracy: %.4f%%' % (g, accuracy))

improvement-07-0.82.hdf5 Test accuracy: 81.5100%
improvement-02-0.73.hdf5 Test accuracy: 71.7100%
improvement-13-0.83.hdf5 Test accuracy: 83.1400%
improvement-01-0.68.hdf5 Test accuracy: 68.1500%
improvement-23-0.84.hdf5 Test accuracy: 83.2500%
improvement-06-0.81.hdf5 Test accuracy: 80.5600%
improvement-16-0.83.hdf5 Test accuracy: 82.9200%
improvement-05-0.80.hdf5 Test accuracy: 80.2400%
improvement-04-0.79.hdf5 Test accuracy: 78.7700%
improvement-10-0.83.hdf5 Test accuracy: 82.7000%
improvement-09-0.82.hdf5 Test accuracy: 82.1300%
improvement-03-0.79.hdf5 Test accuracy: 77.8900%
